In [1]:
%matplotlib inline
from __future__ import print_function
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import LSTM, SpatialDropout1D 
from sklearn.model_selection import train_test_split
import csv
import keras
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import os
#from getcpuinfo import cpuinfo
from keras import regularizers
import pandas
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import requests
from apscheduler.schedulers.background import BackgroundScheduler
import datetime
import sys, os
sys.path.insert(0, os.path.abspath('..'))
from apis.getcpuinfo import cpuinfo 
import time
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


In [2]:
def getTrainedLSTMModel(maxLengthParam):
    
    model = Sequential()
    
    #model.add(SpatialDropout1D(0.3))
    
    model.add(LSTM((50),dropout = 0.3,batch_input_shape = (None,maxLengthParam,1), return_sequences = True))
    
    model.add(Dense(50, activation = 'tanh'))
    
    #model.add(SpatialDropout1D(0.3))
    
    model.add(LSTM(25,return_sequences = False))
    
    model.add(Dense(3, activation='softmax'))
    
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',metrics = ['accuracy'])
    
    if os.path.isfile("LSTMBestWeights.hdf5"):
        
        #print("Hey")
        
        model.load_weights("LSTMBestWeights.hdf5")
        
        return model
    else:
        
        print("Unable to find trained model")
        
        return -1;
    
    
def getTrained3OuptutModel(maxLengthParam):
    
    model = Sequential()
    
    #model.add(SpatialDropout1D(0.3))
    
    model.add(Flatten(batch_input_shape = (None,maxLengthParam,1)))
    
    model.add(Dense(50, activation = "relu"))
    
    model.add(Dropout(0.3, noise_shape=None, seed=None))
    
    model.add(Dense(25, activation = 'relu'))
    
    model.add(Dropout(0.3, noise_shape=None, seed=None))
    
    #model.add(SpatialDropout1D(0.3))
    
    model.add(Dense(12,activation = 'relu'))
    
    model.add(Dropout(0.3, noise_shape=None, seed=None))
    
    model.add(Dense(6,activation = 'relu'))
    
    model.add(Dropout(0.3, noise_shape=None, seed=None))
    
    model.add(Dense(3, activation='softmax'))
    
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',metrics = ['accuracy'])
    
    if os.path.isfile("3OutputDNNBestWeights.hdf5"):
        
        #print("Hey")
        
        model.load_weights("3OutputDNNBestWeights.hdf5")
        
        print("Returning Trained Model")
        
        return model
    else:
        
        print("Unable to find trained model")
        
        return -1;
    
def getTrainedRL3OutputDNN(historyParam):
    
    usageInputLayer = keras.layers.Input(shape=(historyParam,1))
    
    flattened_layer = keras.layers.Flatten()(usageInputLayer)
    
    fullyConnected_0 = keras.layers.Dense(125, activation = 'relu',
                    kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.1))(flattened_layer)
    
    fullyConnected_1 = keras.layers.Dense(100, activation = 'relu',
                    kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.1))(fullyConnected_0)
    
    #drop1 = keras.layers.Dropout(0.25)(fullyConnected_1)
    fullyConnected_2 = keras.layers.Dense(75, activation = 'relu',
                    kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.1))(fullyConnected_1)
    
    #drop2 = keras.layers.Dropout(0.25)(fullyConnected_2)
    fullyConnected_3 = keras.layers.Dense(50, activation = 'relu',
                    kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.1))(fullyConnected_2)
    
    fullyConnected_4 = keras.layers.Dense(25, activation = 'relu',
                    kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.1))(fullyConnected_3)
    
    fullyConnected_5 = keras.layers.Dense(12, activation = 'relu',
                    kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(0.1))(fullyConnected_4)
    
    fullyConnected_6 = keras.layers.Dense(6, activation = 'relu',
                    kernel_initializer='he_normal')(fullyConnected_5)
    
    softmax_output = keras.layers.Dense(3,activation='softmax')(fullyConnected_5)
    
    predictionModel = keras.models.Model(inputs=usageInputLayer,outputs=softmax_output)
    
    if os.path.isfile("3OutputRL3BestWeights.hdf5"):
        
        #print("Hey")
        
        predictionModel.load_weights("3OutputRL3BestWeights.hdf5")
        
        print("Returning Trained Model")
        
        return predictionModel
    
    else:
        
        print("Unable to find trained model")
        
        return -1;
    
def getTrainedRLLSTM(historyParam):
    
    usageInputLayer = keras.layers.Input(shape=(historyParam,1))
    
    #flattened_layer = keras.layers.Flatten()(usageInputLayer)
    
    LSTM_1 = keras.layers.LSTM((50), return_sequences = True)(usageInputLayer)
    
    full_connect_2 = keras.layers.Dense(25, activation = 'relu',kernel_initializer='he_normal')(LSTM_1)
    
    LSTM_3 = keras.layers.LSTM(25, return_sequences = False)(full_connect_2)
    
    full_connect_3 = keras.layers.Dense(12, activation = 'relu',kernel_initializer='he_normal')(LSTM_3)
    
    full_connect_4 = keras.layers.Dense(6, activation = 'relu',kernel_initializer='he_normal')(full_connect_3)
    
    softmax_output = keras.layers.Dense(3,activation='softmax',kernel_initializer='he_normal')(full_connect_4)
    
    predictionModel = keras.models.Model(inputs=usageInputLayer,outputs=softmax_output)
    
    if os.path.isfile("LSTMV3RLBestWeights.hdf5"):
        
        #print("Hey")
        
        predictionModel.load_weights("LSTMV3RLBestWeights.hdf5")
        
        print("Returning Trained Model")
        
        return predictionModel
    else:
        
        print("Unable to find trained model")
        
        return -1



In [3]:
def appendToListOld(oldList,newList,maxLength):
    
    #append to the new entries
    oldList = np.insert(oldList,0,np.array([newList]),axis = 0)
    
    #Remove oldest row if there are more than 50 entries
    if(oldList.shape[0]>maxLength):
        oldList = oldList[:49,:]

    return oldList


def appendToList(oldList,newList,maxLength):
    
    #append to the new entries
    #oldList = np.insert(oldList,0,np.array([newList]),axis = 0)
    oldList = np.append(oldList,np.array([newList]),axis = 0)
    #Remove oldest row if there are more than 50 entries
    if(oldList.shape[0]>maxLength):
        oldList = oldList[1:,:]
    #print(oldList)
    return oldList

def getPredictionsForOneTimeStep(model,usageList,maxLengthParam):
    
    currUsageList = cpuinfo()
    
    updatedList = appendToList(usageList,currUsageList,maxLengthParam)
    
    predictions = []
    
    predDict = {0:0,1:1,2:-1}
    
    for i in range(updatedList.shape[1]):
        
        if updatedList.shape[0]==maxLengthParam:
            
            inputArray = np.expand_dims(np.expand_dims(updatedList[:,i],axis = 1),axis = 0)
            
            modelOutput = model.predict(inputArray)
            
            print("InputArray: ", inputArray)
            
            print("Model Output: ",modelOutput)

            modelPrediction = predDict[np.argmax(modelOutput)]

            predictions.append(modelPrediction)
        
        else:
            
            predictions.append(0)
    
    return predictions, updatedList

#overProvisionedList = 

def getRightOutputs(usageList,minThreshold, maxThreshold):
    
    currList = usageList[0,:]
    print(currList)
    
    target = []
    
    for i in range(usageList.shape[1]):
        if currList[i] > maxThreshold:# Scale Up
            #print("MaxThres ",maxThreshold," ",currList[i])
            target.append(1)
        elif currList[i] < minThreshold: # Scale Down
            #print("MinThres ",minThreshold," ",currList[i])
            target.append(-1)
        else:
            #print("Same ",currList[i])
            target.append(0)  #Remain same
    
    return target

        

def callAPIWithPredictions(predictions,URL):

    headers = {
        'Content-Type': 'application/json',
    }

    data = '{"predictions":['+ ','.join(str(num) for num in predictions) +']}'
    print(data)
    response = requests.post(URL, headers=headers, data=data)

In [4]:
def generatePredictions(model, maxLengthParam,containerCount,minThreshold, maxThreshold):
    
    usageArray = np.empty(shape = (0,containerCount))
    
    cnt = 0;
    
    correctOutputArray = np.empty(shape = (0,containerCount))
    
    predictionsArray = np.empty(shape = (0,containerCount))
    
    while cnt<80:
        
        time.sleep(2)
        
        predictions,usageArray = getPredictionsForOneTimeStep(model,usageArray,maxLengthParam)
        
        predictionsArray = appendToList(predictionsArray, predictions, maxLengthParam)
        
        
        if(usageArray.shape[0]==maxLengthParam):
    
            correctOutputsList = getRightOutputs(usageArray,minThreshold, maxThreshold)

            correctOutputArray = appendToList(correctOutputArray, correctOutputsList, maxLengthParam)
            
            #print("Predictions: ",predictions)
            
            #print("Expected Outputs: ",correctOutputsList)
            
            #callAPIWithPredictions(predictions,"http://152.46.19.96:5000/changeVCPU")
            
            print(accuracy_score(predictions,correctOutputsList))
            
            #percentArray = getPercentagesBasedOnPredictions(predictions,usageArray,minThreshold, maxThreshold)
            
            #print("Percentage Changes",percentArray)
            #for i in range(usageArray.shape[1]):

                #correctOutputsForTraining = to_categorical(correctOutputs,num_classes = 3)
        print("Predictions: ",predictions)
        
        cnt+=1

In [5]:
LSTMModel = getTrainedLSTMModel(50)
RL3OutputDNN = getTrainedRL3OutputDNN(50)
RLLSTMModel = getTrainedRLLSTM(50)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Returning Trained Model
Returning Trained Model


In [6]:
temp = cpuinfo()
test = generatePredictions(RLLSTMModel,50,len(temp),30,70)

Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
Predictions:  [0, 0, 0, 0]
P

InputArray:  [[[ 40.8749012]
  [ 29.1920268]
  [ 59.3405506]
  [ 19.3340013]
  [ 43.106872 ]
  [ 82.9928306]
  [ 78.5092853]
  [ 50.3467386]
  [ 40.2642499]
  [ 59.5544275]
  [ 24.1187681]
  [ 44.4092961]
  [ 60.9368405]
  [ 12.7149189]
  [ 42.0806603]
  [ 58.7107092]
  [ 83.698088 ]
  [ 26.2360032]
  [ 69.4382962]
  [ 23.6727654]
  [ 51.1820636]
  [ 56.2301539]
  [102.1734561]
  [ 20.8063127]
  [ 41.7858595]
  [ 59.7576845]
  [ 66.5474629]
  [ 28.5778999]
  [ 82.8082614]
  [ 64.2610844]
  [ 35.6757254]
  [107.0103094]
  [ 34.9795867]
  [ 22.2152587]
  [ 10.4402473]
  [ 51.4659648]
  [ 37.6543563]
  [ 97.7738731]
  [ 56.0874722]
  [ 40.1909285]
  [ 23.3511141]
  [ 49.3383733]
  [ 43.1415345]
  [ 58.6628009]
  [ 44.3937635]
  [ 69.1624914]
  [ 78.7192933]
  [ 41.3578648]
  [ 33.4573935]
  [ 36.2291506]]]
Model Output:  [[0.9878621  0.00336825 0.00876969]]
InputArray:  [[[18.7212223]
  [19.2090812]
  [51.9256975]
  [17.5998717]
  [17.3419731]
  [21.144607 ]
  [17.3581249]
  [41.4096187]


InputArray:  [[[ 19.3340013]
  [ 43.106872 ]
  [ 82.9928306]
  [ 78.5092853]
  [ 50.3467386]
  [ 40.2642499]
  [ 59.5544275]
  [ 24.1187681]
  [ 44.4092961]
  [ 60.9368405]
  [ 12.7149189]
  [ 42.0806603]
  [ 58.7107092]
  [ 83.698088 ]
  [ 26.2360032]
  [ 69.4382962]
  [ 23.6727654]
  [ 51.1820636]
  [ 56.2301539]
  [102.1734561]
  [ 20.8063127]
  [ 41.7858595]
  [ 59.7576845]
  [ 66.5474629]
  [ 28.5778999]
  [ 82.8082614]
  [ 64.2610844]
  [ 35.6757254]
  [107.0103094]
  [ 34.9795867]
  [ 22.2152587]
  [ 10.4402473]
  [ 51.4659648]
  [ 37.6543563]
  [ 97.7738731]
  [ 56.0874722]
  [ 40.1909285]
  [ 23.3511141]
  [ 49.3383733]
  [ 43.1415345]
  [ 58.6628009]
  [ 44.3937635]
  [ 69.1624914]
  [ 78.7192933]
  [ 41.3578648]
  [ 33.4573935]
  [ 36.2291506]
  [ 54.0791865]
  [ 29.526775 ]
  [ 64.5021398]]]
Model Output:  [[0.98514634 0.00394758 0.0109061 ]]
InputArray:  [[[17.5998717]
  [17.3419731]
  [21.144607 ]
  [17.3581249]
  [41.4096187]
  [16.7932389]
  [16.562492 ]
  [22.8560828]


InputArray:  [[[ 78.5092853]
  [ 50.3467386]
  [ 40.2642499]
  [ 59.5544275]
  [ 24.1187681]
  [ 44.4092961]
  [ 60.9368405]
  [ 12.7149189]
  [ 42.0806603]
  [ 58.7107092]
  [ 83.698088 ]
  [ 26.2360032]
  [ 69.4382962]
  [ 23.6727654]
  [ 51.1820636]
  [ 56.2301539]
  [102.1734561]
  [ 20.8063127]
  [ 41.7858595]
  [ 59.7576845]
  [ 66.5474629]
  [ 28.5778999]
  [ 82.8082614]
  [ 64.2610844]
  [ 35.6757254]
  [107.0103094]
  [ 34.9795867]
  [ 22.2152587]
  [ 10.4402473]
  [ 51.4659648]
  [ 37.6543563]
  [ 97.7738731]
  [ 56.0874722]
  [ 40.1909285]
  [ 23.3511141]
  [ 49.3383733]
  [ 43.1415345]
  [ 58.6628009]
  [ 44.3937635]
  [ 69.1624914]
  [ 78.7192933]
  [ 41.3578648]
  [ 33.4573935]
  [ 36.2291506]
  [ 54.0791865]
  [ 29.526775 ]
  [ 64.5021398]
  [ 47.328447 ]
  [ 47.7805502]
  [ 41.4040426]]]
Model Output:  [[0.9909237  0.00273004 0.00634626]]
InputArray:  [[[17.3581249]
  [41.4096187]
  [16.7932389]
  [16.562492 ]
  [22.8560828]
  [15.630454 ]
  [16.2820129]
  [14.6778416]


InputArray:  [[[ 59.5544275]
  [ 24.1187681]
  [ 44.4092961]
  [ 60.9368405]
  [ 12.7149189]
  [ 42.0806603]
  [ 58.7107092]
  [ 83.698088 ]
  [ 26.2360032]
  [ 69.4382962]
  [ 23.6727654]
  [ 51.1820636]
  [ 56.2301539]
  [102.1734561]
  [ 20.8063127]
  [ 41.7858595]
  [ 59.7576845]
  [ 66.5474629]
  [ 28.5778999]
  [ 82.8082614]
  [ 64.2610844]
  [ 35.6757254]
  [107.0103094]
  [ 34.9795867]
  [ 22.2152587]
  [ 10.4402473]
  [ 51.4659648]
  [ 37.6543563]
  [ 97.7738731]
  [ 56.0874722]
  [ 40.1909285]
  [ 23.3511141]
  [ 49.3383733]
  [ 43.1415345]
  [ 58.6628009]
  [ 44.3937635]
  [ 69.1624914]
  [ 78.7192933]
  [ 41.3578648]
  [ 33.4573935]
  [ 36.2291506]
  [ 54.0791865]
  [ 29.526775 ]
  [ 64.5021398]
  [ 47.328447 ]
  [ 47.7805502]
  [ 41.4040426]
  [ 36.9359763]
  [ 56.7076289]
  [ 25.2392922]]]
Model Output:  [[0.02801665 0.02577366 0.9462097 ]]
InputArray:  [[[16.562492 ]
  [22.8560828]
  [15.630454 ]
  [16.2820129]
  [14.6778416]
  [14.3459795]
  [17.4083002]
  [47.7654668]


...

InputArray:  [[[ 58.7107092]
  [ 83.698088 ]
  [ 26.2360032]
  [ 69.4382962]
  [ 23.6727654]
  [ 51.1820636]
  [ 56.2301539]
  [102.1734561]
  [ 20.8063127]
  [ 41.7858595]
  [ 59.7576845]
  [ 66.5474629]
  [ 28.5778999]
  [ 82.8082614]
  [ 64.2610844]
  [ 35.6757254]
  [107.0103094]
  [ 34.9795867]
  [ 22.2152587]
  [ 10.4402473]
  [ 51.4659648]
  [ 37.6543563]
  [ 97.7738731]
  [ 56.0874722]
  [ 40.1909285]
  [ 23.3511141]
  [ 49.3383733]
  [ 43.1415345]
  [ 58.6628009]
  [ 44.3937635]
  [ 69.1624914]
  [ 78.7192933]
  [ 41.3578648]
  [ 33.4573935]
  [ 36.2291506]
  [ 54.0791865]
  [ 29.526775 ]
  [ 64.5021398]
  [ 47.328447 ]
  [ 47.7805502]
  [ 41.4040426]
  [ 36.9359763]
  [ 56.7076289]
  [ 25.2392922]
  [ 57.2501976]
  [ 82.577119 ]
  [ 54.2247971]
  [ 49.4906289]
  [ 65.4192046]
  [ 71.56567  ]]]
Model Output:  [[0.99074614 0.00285244 0.00640152]]
InputArray:  [[[17.4083002]
  [47.7654668]
  [15.082627 ]
  [18.3644575]
  [13.6224567]
  [13.6090981]
  [43.6736935]
  [18.2847472]


InputArray:  [[[ 69.4382962]
  [ 23.6727654]
  [ 51.1820636]
  [ 56.2301539]
  [102.1734561]
  [ 20.8063127]
  [ 41.7858595]
  [ 59.7576845]
  [ 66.5474629]
  [ 28.5778999]
  [ 82.8082614]
  [ 64.2610844]
  [ 35.6757254]
  [107.0103094]
  [ 34.9795867]
  [ 22.2152587]
  [ 10.4402473]
  [ 51.4659648]
  [ 37.6543563]
  [ 97.7738731]
  [ 56.0874722]
  [ 40.1909285]
  [ 23.3511141]
  [ 49.3383733]
  [ 43.1415345]
  [ 58.6628009]
  [ 44.3937635]
  [ 69.1624914]
  [ 78.7192933]
  [ 41.3578648]
  [ 33.4573935]
  [ 36.2291506]
  [ 54.0791865]
  [ 29.526775 ]
  [ 64.5021398]
  [ 47.328447 ]
  [ 47.7805502]
  [ 41.4040426]
  [ 36.9359763]
  [ 56.7076289]
  [ 25.2392922]
  [ 57.2501976]
  [ 82.577119 ]
  [ 54.2247971]
  [ 49.4906289]
  [ 65.4192046]
  [ 71.56567  ]
  [ 60.4855879]
  [ 40.2356185]
  [ 69.009169 ]]]
Model Output:  [[0.99080044 0.00283777 0.0063618 ]]
InputArray:  [[[18.3644575]
  [13.6224567]
  [13.6090981]
  [43.6736935]
  [18.2847472]
  [15.2284147]
  [19.7837527]
  [41.4274246]


InputArray:  [[[ 56.2301539]
  [102.1734561]
  [ 20.8063127]
  [ 41.7858595]
  [ 59.7576845]
  [ 66.5474629]
  [ 28.5778999]
  [ 82.8082614]
  [ 64.2610844]
  [ 35.6757254]
  [107.0103094]
  [ 34.9795867]
  [ 22.2152587]
  [ 10.4402473]
  [ 51.4659648]
  [ 37.6543563]
  [ 97.7738731]
  [ 56.0874722]
  [ 40.1909285]
  [ 23.3511141]
  [ 49.3383733]
  [ 43.1415345]
  [ 58.6628009]
  [ 44.3937635]
  [ 69.1624914]
  [ 78.7192933]
  [ 41.3578648]
  [ 33.4573935]
  [ 36.2291506]
  [ 54.0791865]
  [ 29.526775 ]
  [ 64.5021398]
  [ 47.328447 ]
  [ 47.7805502]
  [ 41.4040426]
  [ 36.9359763]
  [ 56.7076289]
  [ 25.2392922]
  [ 57.2501976]
  [ 82.577119 ]
  [ 54.2247971]
  [ 49.4906289]
  [ 65.4192046]
  [ 71.56567  ]
  [ 60.4855879]
  [ 40.2356185]
  [ 69.009169 ]
  [ 61.5520334]
  [ 51.9631049]
  [ 25.3186943]]]
Model Output:  [[0.02817458 0.02604975 0.9457756 ]]
InputArray:  [[[43.6736935]
  [18.2847472]
  [15.2284147]
  [19.7837527]
  [41.4274246]
  [16.9332405]
  [14.1868861]
  [14.9085635]


InputArray:  [[[ 41.7858595]
  [ 59.7576845]
  [ 66.5474629]
  [ 28.5778999]
  [ 82.8082614]
  [ 64.2610844]
  [ 35.6757254]
  [107.0103094]
  [ 34.9795867]
  [ 22.2152587]
  [ 10.4402473]
  [ 51.4659648]
  [ 37.6543563]
  [ 97.7738731]
  [ 56.0874722]
  [ 40.1909285]
  [ 23.3511141]
  [ 49.3383733]
  [ 43.1415345]
  [ 58.6628009]
  [ 44.3937635]
  [ 69.1624914]
  [ 78.7192933]
  [ 41.3578648]
  [ 33.4573935]
  [ 36.2291506]
  [ 54.0791865]
  [ 29.526775 ]
  [ 64.5021398]
  [ 47.328447 ]
  [ 47.7805502]
  [ 41.4040426]
  [ 36.9359763]
  [ 56.7076289]
  [ 25.2392922]
  [ 57.2501976]
  [ 82.577119 ]
  [ 54.2247971]
  [ 49.4906289]
  [ 65.4192046]
  [ 71.56567  ]
  [ 60.4855879]
  [ 40.2356185]
  [ 69.009169 ]
  [ 61.5520334]
  [ 51.9631049]
  [ 25.3186943]
  [ 32.3018078]
  [ 30.9153859]
  [ 43.7198184]]]
Model Output:  [[0.9900081  0.00283952 0.00715236]]
InputArray:  [[[19.7837527]
  [41.4274246]
  [16.9332405]
  [14.1868861]
  [14.9085635]
  [22.5084442]
  [47.9883303]
  [14.5580461]


InputArray:  [[[ 28.5778999]
  [ 82.8082614]
  [ 64.2610844]
  [ 35.6757254]
  [107.0103094]
  [ 34.9795867]
  [ 22.2152587]
  [ 10.4402473]
  [ 51.4659648]
  [ 37.6543563]
  [ 97.7738731]
  [ 56.0874722]
  [ 40.1909285]
  [ 23.3511141]
  [ 49.3383733]
  [ 43.1415345]
  [ 58.6628009]
  [ 44.3937635]
  [ 69.1624914]
  [ 78.7192933]
  [ 41.3578648]
  [ 33.4573935]
  [ 36.2291506]
  [ 54.0791865]
  [ 29.526775 ]
  [ 64.5021398]
  [ 47.328447 ]
  [ 47.7805502]
  [ 41.4040426]
  [ 36.9359763]
  [ 56.7076289]
  [ 25.2392922]
  [ 57.2501976]
  [ 82.577119 ]
  [ 54.2247971]
  [ 49.4906289]
  [ 65.4192046]
  [ 71.56567  ]
  [ 60.4855879]
  [ 40.2356185]
  [ 69.009169 ]
  [ 61.5520334]
  [ 51.9631049]
  [ 25.3186943]
  [ 32.3018078]
  [ 30.9153859]
  [ 43.7198184]
  [ 28.2866562]
  [ 36.5534036]
  [ 44.790074 ]]]
Model Output:  [[0.9903488  0.00278683 0.00686436]]
InputArray:  [[[14.1868861]
  [14.9085635]
  [22.5084442]
  [47.9883303]
  [14.5580461]
  [12.3495368]
  [15.2395643]
  [13.3485627]


In [ ]:
class Prediction():
    def __init__(self, model, maxLengthParam,containerCount,minThreshold, maxThreshold):
        self.usageArray = np.empty(shape = (0,containerCount))
        self.correctOutputArray = np.empty(shape = (0,containerCount))
        self.predictionsArray = np.empty(shape = (0,containerCount))
        self.model = model
        self.maxLengthParam = maxLengthParam
        self.minThreshold = minThreshold
        self.maxThreshold = maxThreshold

    def generatePredictions(self): 
        predictions,self.usageArray = getPredictionsForOneTimeStep(self.model,self.usageArray,self.maxLengthParam)       
        self.predictionsArray = appendToList(self.predictionsArray, predictions, self.maxLengthParam)
        if(self.usageArray.shape[0]==self.maxLengthParam):
            correctOutputsList = getRightOutputs(self.usageArray,self.minThreshold, self.maxThreshold)
            self.correctOutputArray = appendToList(self.correctOutputArray, correctOutputsList, self.maxLengthParam)
            print("Predictions: ",predictions)        
            print("Expected Outputs: ",correctOutputsList)            
            callAPIWithPredictions(predictions,"http://152.46.19.96:5000/changeVCPU")
            print(accuracy_score(predictions,correctOutputsList))

In [ ]:
temp = cpuinfo()
test = Prediction(LSTMModel,50,len(temp),30,70)
for i in range(50):
    test.generatePredictions()

In [ ]:
sched = BackgroundScheduler()
sched.add_job(test.generatePredictions, 'interval', seconds=2)
sched.start()
print(datetime.datetime.now())

In [ ]:
sched.shutdown() # shutdown the scheduler

In [ ]:
mylist = np.array([cpuinfo()])
print(mylist)
print(mylist.shape)

In [ ]:
mylist2 = np.insert(mylist,0, np.array([cpuinfo()]), axis = 0)
print(mylist2)

In [ ]:
mylist = np.append(np.empty(shape = (0,5)), np.array([cpuinfo()]), axis = 0)
print(mylist)
print(mylist.shape)

In [ ]:
mylist[:,1]

In [ ]:
a = [1,2,3,4]
a[1:]
','.join(str(e) for e in a)

In [ ]:
b = np.random.randint(1,100,size = (10,5))
b

In [ ]:
np.expand_dims(np.expand_dims(b[:,1],axis = 1),axis = 0).shape
#b[:,1]

In [ ]:
np.zeros(5)

In [ ]:
cpuinfo()